In [ ]:
import ccxt
import os
import datetime

from dotenv import dotenv_values
from pprint import pprint
from arch import arch_model

import ccxt.async_support as ccxt # link against the asynchronous version of ccxt

In [ ]:
from functions.BS_pricer import BS_pricer
from functions.Parameters import Option_param
from functions.Processes import Diffusion_process

import numpy as np
import scipy as scp
import scipy.stats as ss
from scipy.integrate import quad
from functools import partial

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
config = {
    **dotenv_values(".env.shared"),  # load shared development variables
    **dotenv_values(".env.secret"),  # load sensitive variables
    **os.environ,  # override loaded values with environment variables
}

exchange = ccxt.delta(config)
exchange.verbose = False 

exchange.check_required_credentials()
print(exchange.requiredCredentials) 

In [ ]:
%%capture --no-display

# markets = await exchange.fetchMarkets() # Full reloading
symbol = "BTC/USDT:USDT"
markets = exchange.load_markets()
balance = exchange.fetch_balance()
currencies = exchange.fetch_currencies()
orderbook = exchange.fetch_order_book(symbol)
l2_orderbook = exchange.fetch_l2_order_book(symbol)
fetchTrades = exchange.fetch_trades(symbol)
ticker = exchange.fetch_ticker(symbol)
positions = exchange.fetch_positions()
ledger = exchange.fetch_ledger()

# risk = exchange.fetch_positions_risk(symbol) # not suppoted yet
# transactions = exchange.fetch_transactions() # not supported yet

In [ ]:
delta_exchange_markets = await exchange.load_markets()
# Retrieve only options contract id
_ = {}
for k, v in delta_exchange_markets.items():
    if k.startswith(symbol): _.update({k:v})

btc_options = {}
for k, v in _.items():
    if v["option"]: btc_options.update({k:v})

In [ ]:
# to get IV/Greeks
delta_exchange_ticker = await exchange.fetchTickers(list(btc_options.keys()))

# to get Index price (Underlying price)
index_info = await exchange.fetch_ticker(symbol, params={'price':'index'})
index_price = index_info['last']

In [ ]:
option_contract = "BTC/USDT:USDT-221130:15900:C"
expiry_hour = 130000
contract = delta_exchange_ticker[option_contract]
pprint(contract)
a = datetime.datetime.now()
#b = datetime.datetime.strptime(contract["expiryDatetime"], "%Y-%m-%dT%H:%M:%SZ")
#seconds_until_expiration = b-a
#seconds_until_expiration

In [ ]:
"""
    Retrieve
    IV
    index price (Underlying price)
    days until expiration
    strike price
    interest rate
    dividend yield (0)
"""
greeks = contract["info"]["greeks"]
ask_iv = float(contract["info"]["quotes"]['ask_iv'])
ask_size = float(contract["info"]["quotes"]['ask_size'])
best_ask = float(contract["info"]["quotes"]['best_ask'])
best_bid = float(contract["info"]["quotes"]['best_bid'])
bid_iv = float(contract["info"]["quotes"]['bid_iv'])
bid_size = float(contract["info"]["quotes"]['bid_size'])
mark_iv = float(contract["info"]["quotes"]['mark_iv'])
strike_price = float(contract["info"]["strike_price"])
spot_price = float(contract["info"]["spot_price"])

In [ ]:
S0=spot_price
K=strike_price
T=1/(365*24)
r=0.1
sigma=ask_iv

In [ ]:
def binomial_price(S0, K, T, r, sigma, N=15000, payoff="call"):
    # N number of periods or number of time steps  

    dT = float(T) / N                             # Delta t
    u = np.exp(sigma * np.sqrt(dT))                 # up factor
    d = 1.0 / u                                   # down factor 

    V = np.zeros(N+1)                             # initialize the price vector
    S_T = np.array( [(S0 * u**j * d**(N - j)) for j in range(N + 1)] )  # price S_T at time T

    a = np.exp(r * dT)    # risk free compounded return
    p = (a - d)/ (u - d)  # risk neutral up probability
    q = 1.0 - p           # risk neutral down probability   

    if payoff =="call":
        V[:] = np.maximum(S_T-K, 0.0)
    else:
        V[:] = np.maximum(K-S_T, 0.0)

    for i in range(N-1, -1, -1):
        V[:-1] = np.exp(-r*dT) * (p * V[1:] + q * V[:-1])    # the price vector is overwritten at each step
            
    return V[0]


In [ ]:
call_price = BS_pricer.BlackScholes("call", S0=S0, K=K, T=T ,r=r, sigma=sigma)
print(call_price) # our price
print(binomial_price(S0,K,T,r,sigma)) # binomial price
print(best_ask) # market price

In [ ]:
import ccxt
import pandas as pd

exch = 'binance' # initial exchange
t_frame = '1d' # 1-day timeframe, usually from 1-minute to 1-week depending on the exchange
symbol = 'BTC/USDT' # initial symbol
#exchange_list = ['binance','bitfinex','kraken','poloniex','gateio','kucoin','okex','okex3','delta']
exchange_list = ['binance','delta', 'kraken', 'poloniex']
 
# Get our Exchange
try:
    exchange = getattr(ccxt, exch)()
except AttributeError:
    print('-'*36,' ERROR ','-'*35)
    print('Exchange "{}" not found. Please check the exchange is supported.'.format(exch))
    print('-'*80)
    quit()
 
# Check if fetching of OHLC Data is supported
if exchange.has["fetchOHLCV"] != True:
    print('-'*36,' ERROR ','-'*35)
    print('{} does not support fetching OHLC data. Please use another  exchange'.format(exch))
    print('-'*80)
    quit()
 
# Check requested timeframe is available. If not return a helpful error.
if (not hasattr(exchange, 'timeframes')) or (t_frame not in exchange.timeframes):
    print('-'*36,' ERROR ','-'*35)
    print('The requested timeframe ({}) is not available from {}\n'.format(t_frame,exch))
    print('Available timeframes are:')
    for key in exchange.timeframes.keys():
        print('  - ' + key)
    print('-'*80)
    quit()
 
# Check if the symbol is available on the Exchange
exchange.load_markets()
if symbol not in exchange.symbols:
    print('-'*36,' ERROR ','-'*35)
    print('The requested symbol ({}) is not available from {}\n'.format(symbol,exch))
    print('Available symbols are:')
    for key in exchange.symbols:
        print('  - ' + key)
    print('-'*80)
    quit()
 
 
# Get data
data = exchange.fetch_ohlcv(symbol, t_frame)
header = ['Timestamp', 'Open', 'High', 'Low', 'Close', 'Volume']
df = pd.DataFrame(data, columns=header).set_index('Timestamp')
df['symbol'] = symbol
syms = [symbol]
filename = '{}.csv'.format(t_frame)

for exch in exchange_list:
    try:
        exchange = getattr(ccxt, exch)()
    except AttributeError:
        print('-'*36,' ERROR ','-'*35)
        print('Exchange "{}" not found. Please check the exchange is supported.'.format(exch))
        print('-'*80)
        quit()
    if exchange.has["fetchOHLCV"] != True:
        print('-'*36,' ERROR ','-'*35)
        print('{} does not support fetching OHLC data. Please use another exchange'.format(exch))
        print('-'*80)
        quit()
    if (not hasattr(exchange, 'timeframes')) or (t_frame not in exchange.timeframes):
        print('-'*36,' ERROR ','-'*35)
        print('The requested timeframe ({}) is not available from {}\n'.format(t_frame,exch))
        print('Available timeframes are:')
        for key in exchange.timeframes.keys():
            print('  - ' + key)
        print('-'*80)
        quit()
    exchange.load_markets()
    for coin in exchange.symbols:
        if coin in syms or coin[-3:] != 'BTC':
            continue
        else:
            try:
                data = exchange.fetch_ohlcv(coin, t_frame)
            except:
                continue
            data_df = pd.DataFrame(data, columns=header).set_index('Timestamp')
            data_df['symbol'] = coin
            # df = df.append(data_df)
            df = pd.concat([df, data_df])
            syms.append(coin)
df.index = df.index/1000 #Timestamp is 1000 times bigger than it should be in this case
df['Date'] = pd.to_datetime(df.index,unit='s')
df.to_csv(filename)

In [ ]:
import ccxt
import pandas as pd
from pprint import pprint

t_frame = '1d' # 1-day timeframe, usually from 1-minute to 1-week depending on the exchange
symbol = 'BTC/USDT:USDT' # initial symbol
# exchange_list = ['binance','delta', 'kraken', 'poloniex']
exchange_list = ['delta']
filename = '{}.csv'.format(t_frame)

for exch in exchange_list:
    print(exch)
    try:
        exchange = getattr(ccxt, exch)()
    except AttributeError:
        print('-'*36,' ERROR ','-'*35)
        print('Exchange "{}" not found. Please check the exchange is supported.'.format(exch))
        print('-'*80)
        quit()
    if exchange.has["fetchOHLCV"] != True:
        print('-'*36,' ERROR ','-'*35)
        print('{} does not support fetching OHLC data. Please use another exchange'.format(exch))
        print('-'*80)
        quit()
    if (not hasattr(exchange, 'timeframes')) or (t_frame not in exchange.timeframes):
        print('-'*36,' ERROR ','-'*35)
        print('The requested timeframe ({}) is not available from {}\n'.format(t_frame,exch))
        print('Available timeframes are:')
        for key in exchange.timeframes.keys():
            print('  - ' + key)
        print('-'*80)
        quit()
    exchange.load_markets()
    header = ['Timestamp', 'Open', 'High', 'Low', 'Close', 'Volume']
    data = exchange.fetch_ohlcv(symbol, t_frame)
    df = pd.DataFrame(data, columns=header).set_index('Timestamp')
    df['symbol'] = symbol
    df['exchange'] = exch

    df.index /= 1000 #Timestamp is 1000 times bigger than it should be in this case
    df['Date'] = pd.to_datetime(df.index, unit='s')
    df.to_csv(f"./data/{exch}_{t_frame}")

In [ ]:
returns = 100 * df["Close"].pct_change().dropna()
returns

In [ ]:
model = arch_model(returns)
res = model.fit()
forecast = res.forecast(horizon=1, reindex=False)
variance_forecast = forecast.variance.iloc[-1][0]
volatility_forecast = np.sqrt(variance_forecast)
annualized_volatility_forecast = volatility_forecast * np.sqrt(252) / 100
# Cette VOL est issu de la prediction du GARCH a comparer avec les IV de l'Option Wheel 
# et trade le mispricing
annualized_volatility_forecast

In [ ]:
call_price = BS_pricer.BlackScholes("call", S0=S0, K=K, T=T ,r=r, sigma=annualized_volatility_forecast)
print(call_price) # our price